In [2]:
!pip install tensorflow
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.2 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 5.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [127]:
import os
import numpy as np
import random

import gym
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [171]:
env_name = 'tria-3d-rl-model-'

t_ini= 70.0; h_ini= 40.0; a_ini= 10.0 

t_min =-40.0; t_max=110; h_min=0.0; h_max=100.0; a_min=0.0; a_max=5000.0

act_state = 2

stat_rand_min = -1.0; stat_rand_max = 1.0

equilibrium_cycles= 60

r1 = -0.25; r2 = -0.5; r3 = 2; nr3 = -2

const_weight_vec  = [1, 1, 1, 1]

d3 = {
     0 : [65.0, 80.0, 50.0, 85.0, 40.0, 90.0], 
     1 : [30.0, 50.0, 20.0, 60.0, 10.0, 70.0], 
     2 : [0.0, 19.0, 200.0, 599.0, 600.0, 2000.0]
    }

d1 = {0: [65.0, 80.0], 1: [30.0, 50.0], 2: [0.0, 20.0]}

ppo_model_timesteps= 20000; neural_model_timesteps=20000; a2c_model_timesteps=20000

ppo_model = env_name + 'ppo'; neural_model = env_name + 'ppo-neural'; a2c_model = env_name + 'a2c'

In [172]:
class TriaEnv(Env):
    
    def __init__(self):
        self.action_space = MultiDiscrete(np.array([act_state,act_state,act_state,act_state,act_state]))
        
        self.observation_space = Box(low=np.array([t_min, h_min, a_min]), high=np.array([t_max, h_max, a_max]), dtype=np.float32)
        
        self.state = [TINI + random.uniform(stat_rand_min, stat_rand_max), HINI + random.uniform(stat_rand_min, stat_rand_max), AINI + random.uniform(stat_rand_min, stat_rand_max)]
        
        #print('^^^', self.state, self.action_space)
        
        self.equilibrium_cycles_len = equilibrium_cycles
        
    def step(self, action):
        
        ap_scaled = [1 if e == 1 else -1 for e in action]  # 0 (off) => -1 and 1 (on) => 1
        
        actionPrime = [a * b for a, b in zip(ap_scaled, const_weight_vec)] 
        
        actionAlgo = [actionPrime[a] - actionPrime[len(actionPrime) -a -1] for a in range(len(actionPrime) // 2)]
        
        actionAlgo.append(actionPrime[len(actionPrime) // 2])                                                              
        
        #print('***',actionAlgo, self.state)
        
        self.state = [a + b for a, b in zip(actionAlgo, self.state)]
        
        #print('&&&', actionAlgo, self.state)
        
        #reduce tria simulation length by 1 second
        self.equilibrium_cycles_len -= 1
        
        reward = [r3 if e >= d3[i][0] and e<= d3[i][1] else r2 if e >= d3[i][2] and e<= d3[i][3] else r1 if e >= d3[i][4] and e <= d3[i][5] else nr3 for i, e in enumerate(self.state)]
        #reward = [r3 if e >= d1[i][0] and e <= d1[i][1] else nr3  for i, e in enumerate(self.state)]

        reward = sum(reward)
        #print('$$$', reward)
            
        if self.equilibrium_cycles_len <= 0:
            terminated = True
        else:
            terminated = False
            
        info = {}
        #print('reward:{} state:{}'.format(reward, self.state))
        return self.state, reward, terminated, False, info
    
    def render(self):
        pass
    
    def reset(self):
        
        self.state =[t_ini + random.uniform(stat_rand_min, stat_rand_max),h_ini + random.uniform(stat_rand_min, stat_rand_max),a_ini + random.uniform(stat_rand_min, stat_rand_max)]
        #print('@@@', self.state)
        self.equilibrium_cycles_len = equilibrium_cycles
        
        return self.state, {}
        

In [173]:
env= TriaEnv()

print("1. Sample observation space: {}".format(env.observation_space.sample()))
print("2. Sample action space     : {}".format(env.action_space.sample()))
print("3. Sample state            : {}".format(env.state))

1. Sample observation space: [ -21.374819   89.28859  3081.3967  ]
2. Sample action space     : [1 1 1 1 1]
3. Sample state            : [69.27895209556792, 40.09892764134538, 10.927419406352]


In [174]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    #print(state)
    terminated = False
    score = 0 #[0,0,0] 
    
    while not terminated:
        #env.render()
        action = env.action_space.sample()
        #print(action, terminated , reward)
        #print(env.step(action))
        next_state, reward, terminated, truncated, info = env.step(action) 
        score += reward #[a + b for a, b in zip(reward, score)]
    print('Episode: {} Score: {}'.format(episode, score))
env.close()

Episode: 1 Score: 292.0
Episode: 2 Score: 312.5
Episode: 3 Score: 93.75
Episode: 4 Score: 172.5
Episode: 5 Score: 312


In [175]:
log_path = os.path.join('training','logs')
log_path

'training/logs'

In [176]:
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [177]:
model.learn(total_timesteps=ppo_model_timesteps)

Logging to training/logs/PPO_4
-----------------------------
| time/              |      |
|    fps             | 138  |
|    iterations      | 1    |
|    time_elapsed    | 14   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 110          |
|    iterations           | 2            |
|    time_elapsed         | 36           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0126291625 |
|    clip_fraction        | 0.138        |
|    clip_range           | 0.2          |
|    entropy_loss         | -3.46        |
|    explained_variance   | 0.000926     |
|    learning_rate        | 0.0003       |
|    loss                 | 1.08e+03     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0148      |
|    value_loss           | 2.46e+03     |
----------------------------

In [178]:
tria_model_path = os.path.join('training','save', ppo_model)

In [179]:
model.save(tria_model_path)

In [180]:
del model

In [181]:
model = PPO.load(tria_model_path, env=env)

In [182]:
evaluate_policy(model, env, n_eval_episodes=20, render=True)

(360.0, 0.0)

In [183]:
env.close()

In [184]:
episodes=10
for episode in range(1, episodes+1):
    observation = env.reset()
    terminated = False
    score = 0
    while not terminated:
        env.render()
        action, _ = model.predict(observation)
        observation, reward, terminated , info = env.step(action)
        score += reward
    print('Episone:{} Score:{}'.format( episode, score))
env.close()  

Episone:1 Score:[282.75]
Episone:2 Score:[285.]
Episone:3 Score:[360.]
Episone:4 Score:[342.5]
Episone:5 Score:[262.5]
Episone:6 Score:[357.5]
Episone:7 Score:[332.5]
Episone:8 Score:[259.5]
Episone:9 Score:[342.5]
Episone:10 Score:[322.5]


# custom neural network injected in PPO model based on tria 3D environment

In [185]:
net_arch = dict(pi=[128,128,128,128], vf=[128,128,128,128])

In [186]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device


In [187]:
model.learn(total_timesteps=neural_model_timesteps)

Logging to training/logs/PPO_5
-----------------------------
| time/              |      |
|    fps             | 92   |
|    iterations      | 1    |
|    time_elapsed    | 22   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 72          |
|    iterations           | 2           |
|    time_elapsed         | 56          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010003677 |
|    clip_fraction        | 0.115       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.46       |
|    explained_variance   | -0.000324   |
|    learning_rate        | 0.0003      |
|    loss                 | 885         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0101     |
|    value_loss           | 2.43e+03    |
-----------------------------------------
---

In [188]:
tria_model_path_neural = os.path.join('training','save',neural_model)
model.save(tria_model_path_neural)

In [189]:
del model

In [190]:
model = PPO.load(tria_model_path_neural, env=env)

In [191]:
evaluate_policy(model, env, n_eval_episodes=20, render=True)

(79.5875, 8.751883725804404)

In [192]:
env.close()

# create A2C network based learing with Tria 3D environment

In [193]:
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import SubprocVecEnv

In [194]:
model = A2C("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [195]:
model.learn(total_timesteps=a2c_model_timesteps)

Logging to training/logs/A2C_2
------------------------------------
| time/                 |          |
|    fps                | 72       |
|    iterations         | 100      |
|    time_elapsed       | 6        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -3.39    |
|    explained_variance | 3.45e-05 |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 60.2     |
|    value_loss         | 371      |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 76        |
|    iterations         | 200       |
|    time_elapsed       | 13        |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -3.21     |
|    explained_variance | -0.000909 |
|    learning_rate      | 0.0007    |
|    n_updates          | 199       |
|    policy_loss        | 15.4   

In [196]:
tria_model_path_a2c = os.path.join('training','save',a2c_model)
model.save(tria_model_path_a2c)

In [197]:
del model

In [198]:
model = A2C.load(tria_model_path_a2c, env=env)

In [199]:
evaluate_policy(model, env, n_eval_episodes=20, render=True)

(360.0, 0.0)

In [200]:
env.close()

In [ ]:
!tensorboard --logdir './training/logs/' --bind_all  # training_log_path

2023-03-07 20:51:36.652546: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 20:51:37.101809: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 20:51:37.101989: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 20:51:40.960059: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-